Julia Adamowicz 120591

Labolatorium 3 -API do serwowania modelu ML, testowanie endpointów i zwracanie predykcji

In [ ]:
from flask import Flask, jsonify, request
import threading
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Zadanie 1

In [ ]:
#Tworzenie aplikacji Flask
app = Flask(__name__)

In [ ]:
#Definicja podstawowego endpointu
@app.route('/')
def home():
    return jsonify({"student": "Julia Adamowicz"})

In [ ]:
def run_app():
    app.run(debug=True, use_reloader=False)

#Uruchamiamy Flask w osobnym wątku
threading.Thread(target=run_app).start()

Zadanie 2

In [ ]:
#Tworzymy przykładowe dane: x (cecha) i y (wartość docelowa)
X = np.array([[1], [2], [3], [4], [5]])  #Przykładowe dane wejściowe
y = np.array([1, 2, 3, 4, 5])  #Wynik 

#Tworzymy model LinearRegression i go trenujemy
model = LinearRegression()
model.fit(X, y)

In [ ]:
#Endpoint /predict przyjmujący dane wejściowe w formacie JSON
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  #Pobieramy dane JSON z żądania
    if 'feature' not in data:
        return jsonify({"error": "Brak wymaganej cechy 'feature' w danych wejściowych"}), 400

    feature_value = data['feature']  # obieramy wartość cechy

    #Sprawdzamy, czy feature_value jest liczbą (float lub int)
    if not isinstance(feature_value, (int, float)):
        return jsonify({"error": "'feature' musi być liczbą"}), 400
    
    feature_array = np.array([[feature_value]])  #Tworzymy tablicę numpy z danymi wejściowymi

    #Predykcja za pomocą modelu
    prediction = model.predict(feature_array)

    #Zwracamy wynik w formacie JSON
    return jsonify({"prediction": prediction[0]})

#Poprawne zapytanie
!curl -X POST http://127.0.0.1:5000/predict -H "Content-Type: application/json" -d "{\"feature\": 6}"

In [ ]:
#Błędne zapytanie (brak cechy)
!curl -X POST http://127.0.0.1:5000/predict -H "Content-Type: application/json" -d "{}"

In [ ]:
#Błędne zapytanie (zły typ danych)
!curl -X POST http://127.0.0.1:5000/predict -H "Content-Type: application/json" -d "{\"feature\": \"abc\"}"

Zadanie 4

In [ ]:
#Endpoint /info zwracający informacje o modelu
@app.route('/info', methods=['GET'])
def info():
    model_info = {
        "model_type": str(type(model)),
        "num_features": model.n_features_in_
    }
    return jsonify(model_info)

In [ ]:
!curl http://127.0.0.1:5000/info

In [ ]:
#Endpoint /health zwracający status serwera
@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "feelin' good"})

In [ ]:
!curl http://127.0.0.1:5000/health

Zadanie 5

In [ ]:
!pip install waitress

In [ ]:
code = """
# -*- coding: utf-8 -*-from flask import Flask, request, jsonify
from flask import Flask, request, jsonify
from sklearn.linear_model import LinearRegression
import numpy as np
from waitress import serve

app = Flask(__name__)

# Przykładowy model ML (tutaj regresja liniowa)
model = LinearRegression()
model.fit(np.array([[1], [2], [3], [4]]), np.array([1, 2, 3, 4]))

@app.route('/')
def home():
    return jsonify({"message":"Hello, World!"})

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    if 'input' not in data:
        return jsonify({"error": "Missing 'input' value"}), 400
    prediction = model.predict(np.array([[data['input']]]))
    return jsonify({"prediction": prediction[0]})

if __name__ == '__main__':
    serve(app, host='0.0.0.0', port=8000)

"""

In [ ]:
with open("app.py", "w") as file:
    file.write(code)

In [34]:
!curl -X POST http://localhost:8000/predict -H "Content-Type: application/json" -d "{\"input\": 5}"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 8000 after 2239 ms: Could not connect to server


In [ ]:
!curl -X POST http://localhost:8000/predict -H "Content-Type: application/json" -d "{}"

Laboratorium 4:  Docker i konteneryzacja modelu ML

In [1]:
%%writefile requirements.txt
flask
numpy
scikit-learn
gunicorn

Overwriting requirements.txt


In [2]:
%%writefile C:\Users\julia\JupyterNotebook\Dockerfile
#Użycie lekkiego obrazu Pythona
FROM python:3.9-slim

#Ustawienie katalogu roboczego
WORKDIR /app

#Kopiowanie plików aplikacji
COPY . /app

#Instalacja zależności
RUN pip install --no-cache-dir -r requirements.txt

#Wystawienie portu
EXPOSE 5000

#Uruchomienie serwera Gunicorn
CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:5000", "app:app"]

Writing C:\Users\julia\JupyterNotebook\Dockerfile


In [3]:
%%writefile C:\Users\julia\JupyterNotebook\app.py
from flask import Flask, request, jsonify
import numpy as np
from sklearn.linear_model import LinearRegression

app = Flask(__name__)

# Przykładowe dane do trenowania modelu ML
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])
model = LinearRegression().fit(X, y)

@app.route("/", methods=["GET"])
def home():
    return jsonify({"student": "Julia Adamowicz"})

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    if "input" not in data:
        return jsonify({"error": "Brak wymaganej wartosci"}), 400
    try:
        input_value = np.array([[data["input"]]])
        prediction = model.predict(input_value).tolist()
        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Overwriting C:\Users\julia\JupyterNotebook\app.py


In [41]:
!curl http://localhost:5000

{"student":"Julia Adamowicz"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    30  100    30    0     0   2550      0 --:--:-- --:--:-- --:--:--  2727


In [42]:
!curl -X POST http://localhost:5000/predict -H "Content-Type: application/json" -d "{\"input\": 5}"

{"prediction":[10.0]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    34  100    22  100    12   1277    697 --:--:-- --:--:-- --:--:--  2000


In [21]:
!pip install redis


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
%%writefile C:\Users\julia\JupyterNotebook\app.py
from flask import Flask, request, jsonify
import numpy as np
import redis
from sklearn.linear_model import LinearRegression

app = Flask(__name__)

# Połączenie z Redis (host "redis" zamiast "localhost", bo Docker używa nazw kontenerów)
redis_client = redis.Redis(host="redis", port=6379, decode_responses=True)

# Przykładowe dane do trenowania modelu ML
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])
model = LinearRegression().fit(X, y)

@app.route("/", methods=["GET"])
def home():
    return jsonify({"student": "Julia Adamowicz"})

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    if "input" not in data:
        return jsonify({"error": "Brak wymaganej wartosci"}), 400
    try:
        input_value = np.array([[data["input"]]])
        prediction = model.predict(input_value).tolist()

        # Zapis wyniku do Redis
        redis_client.set("last_prediction", prediction[0])

        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

@app.route("/last", methods=["GET"])
def last_prediction():
    """ Pobiera ostatnią predykcję z Redis """
    last_pred = redis_client.get("last_prediction")
    if last_pred is None:
        return jsonify({"error": "Brak zapisanej predykcji"}), 404
    return jsonify({"last_prediction": float(last_pred)})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Overwriting C:\Users\julia\JupyterNotebook\app.py


In [29]:
!curl http://localhost:5000/

{"student":"Julia Adamowicz"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    30  100    30    0     0   2427      0 --:--:-- --:--:-- --:--:--  2307


In [30]:
!curl -X POST http://localhost:5000/predict -H "Content-Type: application/json" -d "{\"input\": 5}"

{"prediction":[10.0]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    34  100    22  100    12   2463   1343 --:--:-- --:--:-- --:--:--  4250


In [43]:
!curl http://localhost:5000/last

{"last_prediction":10.0}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    25  100    25    0     0   1241      0 --:--:-- --:--:-- --:--:--  1250
